In [82]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio import features
from glob import glob
from tqdm import tqdm

In [8]:
# Combining the two into one dataframe
def merge_shp_bnd(shp,bbnd):
    shp = shp.set_index("GEOID10")
    shp.index = shp.index.map(int)
    
    bbnd = bbnd.set_index("FIPS")
    bbnd.index = bbnd.index.map(int)
    return shp.merge(bbnd,left_index=True, right_index=True, how="inner")

def lteraster_from_sat(lte, path):
    with rio.open(path) as dataset:
        
        satimg = dataset.read()
        mask = dataset.dataset_mask()
        sat_trs = dataset.transform
        sat_crs = dataset.crs

        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                rio.features.shapes(mask, transform=dataset.transform)))

    sat = gpd.GeoDataFrame.from_features(list(results),crs=sat_crs)
    
    lte = lte.to_crs(sat_crs)
    ovr = gpd.overlay(sat,lte)
    
    shapes = [(geom,value) for geom, value in zip(lte["geometry"],lte["4G_LTE"])]
    return features.rasterize(shapes,np.shape(satimg[0]),transform=sat_trs)


In [10]:
%%time
actual477 = pd.read_csv("./Data/fcc477actual/CA_201906.csv")
censusblocks = gpd.read_file("./Data/shapefiles/censusblocks/ca/tl_2019_06_tabblock10.shp")

Wall time: 55.3 s


In [11]:
%%time
merged = merge_shp_bnd(censusblocks,actual477)

Wall time: 6.01 s


In [ ]:
open_path = glob("./Data/sentinel2/ca/composite/*.tif")
save_path = "./Data/fcc477actual/sentinel2/ca/"

for path in tqdm(open_path):
    sat_id = path.split("\\")[1].split(".")[0]
    lteraster = lteraster_from_sat(merged, path)
    np.savez_compressed(save_path+sat_id,lteraster)

  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\Tyler\anaconda3\envs\cs230\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
  5%|▌         | 1/20 [03:39<1:09:30, 219.50s/it]C:\Users\Tyler\anaconda3\envs\cs230\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
 10%|█         | 2/20 [07:33<1:07:11, 223.97s/it]C:\Users\Tyler\anaconda3\envs\cs230\lib\site-packages\pyproj\crs\crs.py:53: FutureWarn

In [85]:
print("hello")

hello
